## Zadanie domowe: morfologiczna gra w życie – John Conway

### Wykorzystanie operacji LUT w przekształceniu trafi, nie trafi
  - Szybszą metodą wykonania transformacji trafi, nie trafi może być operacja LUT.
  - Technika polega na zakodowaniu wyników wszystkich interesujących  konfiguracji, a następnie podczas przetwarzania wykorzystania operacji LUT.
  - Dla otoczenia 3x3 możliwe jest 512 różnych konfiguracji.
  - Aby praktycznie zrealizować operacje, każdej konfiguracji należy nadać unikalny indeks. Jedną z możliwości jest pomnożenie elementu strukturalnego przez macierz (mnożenie odpowiednich elementów):
  ```
  [[1, 8,  64],
   [ 2, 16, 128],
   [ 4, 32, 256]]
  ```
  Przykładowo elementowi:
  ```
  [[1, 1, 0],
   [ 1, 0, 1],
   [ 1, 0, 1]]
  ```
  odpowiada liczba: 1(1) + 2(1) + 4(1) + 8(1) + 128(1) + 256(1) = 399.
  
### Przykład działania metody – detekcja punktów końcowych na obrazie.
  - założenie: punkt końcowy to punkt, który ma dokładnie jednego sąsiada,
  - zdefiniuj funkcję, która jako argument pobiera otoczenie, a zwraca 0 lub 1 w zależności od tego, czy rozpatrywany punkt jest końcowy np. dla sąsiedztwa 3×3 punkt będzie końcowy, jeżeli jest zapalony i ma tylko jednego sąsiada (czyli suma pikseli jest równa 2).
  - wygeneruj przekodowanie LUT.
  - wczytaj obraz szkielet.bmp (należy go przekształcić, aby uzyskać dwuwymiarową tablicę o wartościach 0-1). Wykorzystując wygenerowane przekodowanie LUT wykonaj detekcję zakończeń. Wyświetl obraz oryginalny i po przekodowaniu LUT.

### Gra w życie

Reguły gry w życie:
  - każdy piksel biały, który ma dwóch lub trzech sąsiadów (białych) przeżywa,
  - każdy piksel biały, który ma 0,1 lub więcej niż trzech sąsiadów (białych) nie przeżywa (głód lub przeludnienie),
  - jeżeli jakieś pole (czarne) sąsiaduje dokładnie z trzema pikselami białymi, to na tym polu ,,rodzi'' się nowy piksel biały.

Zadanie:
  - za pomocą mechanizmu LUT (opisanego wcześniej) należy zaimplementować morfologiczną gre w życie,
  - najważniejszym elementem jest funkcja opisująca reguły gry,
  - symulacje należny przeprowadzić dla plansz dostarczonych w pliku gra.py,
  - dobrze jest wykonać kilka iteracji – zobaczyć jak zmienia się kształt,
  - inne ciekawe kształty do znalezienia w internecie.


In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def generate_endpoint_lut():
    lut = np.zeros(512, dtype=np.uint8)
    for i in range(512):
        binary = f'{i:09b}'
        if binary[4] == '1' and binary.count('1') == 2:
            lut[i] = 1
    return lut

def generate_game_of_life_lut():
    lut = np.zeros(512, dtype=np.uint8)
    for i in range(512):
        binary = f'{i:09b}'
        center = int(binary[4])
        neighbors = binary.count('1') - center
        if center == 1:
            if neighbors == 2 or neighbors == 3:
                lut[i] = 1
            else:
                lut[i] = 0
        else:
            if neighbors == 3:
                lut[i] = 1
            else:
                lut[i] = 0
    return lut

def neighborhood_to_index(neighborhood):
    weights = np.array([[1, 8, 64], [2, 16, 128], [4, 32, 256]])
    return int(np.sum(neighborhood * weights))

def load_image_as_binary(filename):
    image = Image.open(filename).convert('L')
    binary_image = np.array(image) // 255
    return binary_image

def detect_endpoints(image, lut):
    padded_image = np.pad(image, pad_width=1, mode='constant', constant_values=0)
    output_image = np.zeros_like(image)
    for i in range(1, padded_image.shape[0] - 1):
        for j in range(1, padded_image.shape[1] - 1):
            neighborhood = padded_image[i-1:i+2, j-1:j+2]
            index = neighborhood_to_index(neighborhood)
            output_image[i-1, j-1] = lut[index]
    return output_image

def game_of_life_step(image, lut):
    padded_image = np.pad(image, pad_width=1, mode='constant', constant_values=0)
    output_image = np.zeros_like(image)
    for i in range(1, padded_image.shape[0] - 1):
        for j in range(1, padded_image.shape[1] - 1):
            neighborhood = padded_image[i-1:i+2, j-1:j+2]
            index = neighborhood_to_index(neighborhood)
            output_image[i-1, j-1] = lut[index]
    return output_image

def display_image(image, title):
    plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

def test_game_of_life(initial_state, iterations, lut):
    current_state = initial_state
    for i in range(iterations):
        display_image(current_state, f'Iteracja {i}')
        current_state = game_of_life_step(current_state, lut)
    display_image(current_state, f'Ostateczny wynik po {iterations} iteracjach')

if __name__ == "__main__":
    endpoint_lut = generate_endpoint_lut()
    gol_lut = generate_game_of_life_lut()
    binary_image = load_image_as_binary('szkielet.bmp')
    endpoint_image = detect_endpoints(binary_image, endpoint_lut)
    display_image(endpoint_image, 'Punkty końcowe wykryte')
    
    initial_states = [
        np.array([[0, 1, 0], [0, 1, 0], [0, 1, 0]]),
        np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]]),
        np.array([[1, 1, 0, 0], [1, 1, 0, 0], [0, 0, 1, 1], [0, 0, 1, 1]]),
        np.zeros((10, 10)),
    ]
    initial_states[3][1, 1] = 1
    initial_states[3][1, 2] = 1
    initial_states[3][2, 1] = 1
    initial_states[3][2, 2] = 1

    for idx, initial_state in enumerate(initial_states):
        display_image(initial_state, f'Stan początkowy {idx}')
        test_game_of_life(initial_state, 10, gol_lut)
